In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms
from google.colab.patches import cv2_imshow

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import kagglehub
path = kagglehub.dataset_download("ambityga/imagenet100")
print("Path to dataset files:", path)

In [ ]:
!ls -l $path/val.X/n01773549

In [ ]:
imagepath = path + '/val.X/' + 'n01773549' +'/' + 'ILSVRC2012_val_00008316.JPEG'

In [ ]:
image = cv2.imread(imagepath)
cv2_imshow(image)

In [ ]:
# load model
vitb8 = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')

In [ ]:
vitb8.eval()

In [ ]:
# preprocessing
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB
image_size = (224, 224)
blob = cv2.dnn.blobFromImage(image, 1.0/255, image_size, swapRB=True, crop=False)
blob[0][0] = (blob[0][0] - 0.485)/0.229
blob[0][1] = (blob[0][1] - 0.456)/0.224
blob[0][2] = (blob[0][2] - 0.406)/0.225
x = torch.tensor(blob) # 1 x 3 x 224 x 224
print(x.min().item(),x.max().item())

In [ ]:
# embed
B, _, h, w = x.shape
x = vitb8.patch_embed(x)  # patch linear embedding by Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
print(x.shape) # 1 x 784 x 768   # 784 = 28 x 28, 224 = 28 x 8, 768 = 12 x 64

In [ ]:
# add CLS
cls_tokens = vitb8.cls_token.expand(B, -1, -1)
x = torch.cat((cls_tokens, x), dim=1) # 1 x 785 x 768

In [ ]:
# add positional encoding
x = x + vitb8.interpolate_pos_encoding(x, h, w) # 1 x 785 x 768

In [ ]:
# encode
attn_maps = []
for block in vitb8.blocks:
    y, attn = block.attn(block.norm1(x))
    attn_maps.append(attn)
    x += y
    x += block.mlp(block.norm2(x))

x = vitb8.norm(x) # 1 x 785 x 768

In [ ]:
# wipeout
features = x[:, 0] # 1 x 768
print(features.shape)
print(features)

In [ ]:
attn_mats.shape

In [ ]:
# visualize attention maps
attn_mat = torch.stack(attn_maps).squeeze(1) # 12 x 12 x 785 x 785
grid_size = int(np.sqrt(attn_mat.size(-1)))
rows = []
for i, attn_heads in enumerate(attn_mat):
    cols = []
    for j, attn_head in enumerate(attn_heads):
        mask = attn_head[0, 1:].pow(0.5).reshape(grid_size,grid_size).detach().cpu().numpy() # 28 x 28
        mask = (np.sqrt(mask)*255).astype(np.uint8)
        mask = cv2.resize(mask, (image.shape[1],image.shape[0]), interpolation=cv2.INTER_NEAREST)
        mask = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
        cv2.rectangle(mask,(0,0),(mask.shape[1],mask.shape[0]),(200,0,0),2)
        cv2.putText(mask,f'H{j}',(mask.shape[1]*4//10,20),0,0.8,(200,0,0))
        cols.append(mask)

    row = cv2.hconcat(cols)
    cv2.putText(row,f'{i}',(5,20),0,0.8,(0,255,0))
    rows.append(row)

disp = cv2.vconcat(rows)
cv2_imshow(disp)